
# **Project 1: Classification of Time Series Data**

Name: Haneen Alsuradi
NetID: hha243

In this project, I will use 1 dimensional convolotional network to perform classification on time series data. Data is taken from Kaggle competition [Surface Type Classification] found in the link below: https://www.kaggle.com/c/career-con-2019/overview

## **Step 1: Data Preparation**

The first step is to import, view and prepare the data. We import the data saved in X_train.csv and y_train.csv. We split the data to train and test as advised. After that, we view the data using the head() command from pandas.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

# read Kaggle datasets
X_train = pd.read_csv('/kaggle/input/career-con-2019/X_train.csv')
y_train = pd.read_csv('/kaggle/input/career-con-2019/y_train.csv')
# split X_train
samples = 20
time_series = 128
start_x = X_train.shape[0] - samples*time_series
X_train_new, X_test_new = X_train.iloc[:start_x], X_train.iloc[start_x:]
# split y_train
start_y = y_train.shape[0] - samples
y_train_new, y_test_new = y_train.iloc[:start_y], y_train.iloc[start_y:]
X_train_new.head(5)

As can be noticed, the first 3 columns are not part of the features and must be dropped. We drop these columns for X_train_new and X_test_new.

In [ ]:
X_train_new=X_train_new.drop(['row_id', 'series_id','measurement_number'], axis=1)
X_test_new=X_test_new.drop(['row_id', 'series_id','measurement_number'], axis=1)


Now, we have a look at the labels of the training data saved in y_train_new. We are only interested in the surface type which is the last column. We drop the other columns from y_train_new.

In [ ]:
y_train_new.head(5)


We drop the first two columns from the y_train_new and y_test_new as they will not be needed.

In [ ]:
y_train_new=y_train_new.drop(['series_id', 'group_id'], axis=1)
y_test_new=y_test_new.drop(['series_id', 'group_id'], axis=1)

Now, we convert the dataframes to a numpy array. We extract the values from the data frame using .values. We print the shape of training and testing data.

In [ ]:
X_train_new=X_train_new.values
X_test_new=X_test_new.values
y_train_new=y_train_new.values
y_test_new=y_test_new.values
print('The size of X_train_new:', X_train_new.shape )
print('The size of X_test_new:', X_test_new.shape )

print('The size of y_train_new:', y_train_new.shape )
print('The size of y_test_new:', y_test_new.shape )

Now, we convert the strings (concrete, tiled, soft_tiels, ..etc.) in y_train_new and y_test_new to integer labels: 0,1,2... etc using the LabelEncorder function. After that we implement one hot coding using the to_categorical function. We have 9 types of surfaces and thus the number of cloumns for y_train_new and y_test_new  will be 9. One hot coding is suitable for 1D conv net models to prevent poor performance or unexpected results (predictions halfway between categories).

In [ ]:
from keras.utils import to_categorical

labelencoder_y = LabelEncoder()

y=np.concatenate([y_train_new,y_test_new])
y=labelencoder_y.fit_transform(y)
y=to_categorical(y)


y_train_new = y[:-20]
y_test_new = y[-20:]
print('The shape of y_train_new:', y_train_new.shape)
print('The shape of y_test_new:', y_test_new.shape)


We think that the orientation of the robot moving across surfaces is not a relevant information to predict the surface type. Instead, the rate of change of the roll, yaw and pitch can serve as better features for prediction. The rate of change can be affected by the way the robot moves which is directly affected by the tyoe of surface the robot is moving on. We first calculate the roll, yaw and pitch from the orientation information using the transformation formulas as shown below. We add these features to X_train_new and X_test_new (separately) and delete the orientation features (W,X,Y,Z). The rate of change in yaw,pitch and roll will be calculated at a later step.

In [ ]:
#FOR TRAIN DATASET
roll=np.zeros([X_train_new.shape[0],1])
pitch=np.zeros([X_train_new.shape[0],1])
yaw=np.zeros([X_train_new.shape[0],1])


for i in range(X_train_new.shape[0]):
  roll[i] = np.arctan2(2*(X_train_new[i,1]*X_train_new[i,2] + X_train_new[i,3]*X_train_new[i,0]),1 - 2*(X_train_new[i,2]*X_train_new[i,2] + X_train_new[i,3]*X_train_new[i,3]))
  pitch[i] = np.arcsin(2*(X_train_new[i,1]*X_train_new[i,3] - X_train_new[i,0]*X_train_new[i,2]))
  yaw[i] = np.arctan2(2*(X_train_new[i,1]*X_train_new[i,0] + X_train_new[i,2]*X_train_new[i,3]),1 - 2*(X_train_new[i,3]*X_train_new[i,3] + X_train_new[i,0]*X_train_new[i,0]))

X_train_new=np.delete(X_train_new,[0,1,2,3], 1)
X_train_new=np.concatenate((roll,pitch,yaw,X_train_new),axis=1)

#FOR TEST DATA SET
roll=np.zeros([X_test_new.shape[0],1])
pitch=np.zeros([X_test_new.shape[0],1])
yaw=np.zeros([X_test_new.shape[0],1])


for i in range(X_test_new.shape[0]):
  roll[i] = np.arctan2(2*(X_test_new[i,1]*X_test_new[i,2] + X_test_new[i,3]*X_test_new[i,0]),1 - 2*(X_test_new[i,2]*X_test_new[i,2] + X_test_new[i,3]*X_test_new[i,3]))
  pitch[i] = np.arcsin(2*(X_test_new[i,1]*X_test_new[i,3] - X_test_new[i,0]*X_test_new[i,2]))
  yaw[i] = np.arctan2(2*(X_test_new[i,1]*X_test_new[i,0] + X_test_new[i,2]*X_test_new[i,3]),1 - 2*(X_test_new[i,3]*X_test_new[i,3] + X_test_new[i,0]*X_test_new[i,0]))

X_test_new=np.delete(X_test_new,[0,1,2,3], 1)
X_test_new=np.concatenate((roll,pitch,yaw,X_test_new),axis=1)


## **Step 2: Building the model**

The data should be reshaped in a 3D matrix to suit the 1D CNN input data shape. The first dimension is for the samples, the second for the timestamp, and the third for the featueres. 

In [ ]:
nfeatures=X_train_new.shape[1]
ntimestamp=128
nsamples=3790
X_3D_train=X_train_new[:,0].reshape(nsamples,ntimestamp)
for i in range(nfeatures-1):
  i=i+1
  r=X_train_new[:,i].reshape(nsamples,ntimestamp)
  X_3D_train=np.dstack((X_3D_train,r))
print('The shape of X_train: ', X_3D_train.shape)

nfeatures=X_test_new.shape[1]
ntimestamp=128
nsamples=20
X_3D_test=X_test_new[:,0].reshape(nsamples,ntimestamp)
for i in range(nfeatures-1):
  i=i+1
  r=X_test_new[:,i].reshape(nsamples,ntimestamp)
  X_3D_test=np.dstack((X_3D_test,r))
print('The shape of X_test: ', X_3D_test.shape)


As mentioned earlier, we need to include the rate of change for the yaw, pitch and roll as they are affected by the type of surface the robot is moving on. We calculated the roll, yaw and pitch earlier and added them to X_train_new and X_test_new. We will replace them with their rate of change instead. They are stored in the first three features. 

In [ ]:
for i in range(2):
    rate = X_3D_train[:,:,i]
    rate_c = np.copy(rate)
    rate_c[:,1:] = rate_c[:,:-1]
    rate = rate - rate_c
    X_3D_train[:,:,i] = rate
    
for i in range(2):
    rate = X_3D_test[:,:,i]
    rate_c = np.copy(rate)
    rate_c[:,1:] = rate_c[:,:-1]
    rate = rate - rate_c
    X_3D_test[:,:,i] = rate

Another feature we think is important is the fft of the time series data. We believe that each surface will cause the robot to vibrate or oscillate with specific frequencies. Thus, we calculate the fft for all the timeseries features in X_train_new and X_test_new (separately) and add the fft of the features to the corresponding matrix.

In [ ]:
from scipy import fftpack
X_3Dfft = np.abs(np.fft.fft(X_3D_train,axis=1))
#freqs = fftpack.fftfreq(len(x)) * f_s
X_3D_train=np.dstack((X_3D_train,X_3Dfft))
print('The size of X_3D: ', X_3D_train.shape)

from scipy import fftpack
X_3Dfft = np.abs(np.fft.fft(X_3D_test,axis=1))
#freqs = fftpack.fftfreq(len(x)) * f_s
X_3D_test=np.dstack((X_3D_test,X_3Dfft))
print('The size of X_3D: ', X_3D_test.shape)

We print the parameters of X_train_new. Number of: (features, timestamps and samples)

In [ ]:
nfeatures=X_3D_train.shape[2]
ntimestamp=X_3D_train.shape[1]
nsamples=X_3D_train.shape[0]

print('Number of features in Xtrain: ', nfeatures)
print('Number of timestamps in Xtrain: ', ntimestamp)
print('Number of samples in Xtrain: ', nsamples)


We standarize X_train_new and X_test_new separately by calculating the mean and standard deviation for each of the feautures across all samples, and then subtracting the mean and dividing by the standard deviation. Standarization can help the conv net to acheive better results by removing any effects resulted from different recording sessions.

In [ ]:
for k in range(nfeatures):
    X_train_m = np.mean(X_3D_train[:,:,k])
    X_train_sd = np.std(X_3D_train[:,:,k])
    X_3D_train[:,:,k] = (X_3D_train[:,:,k]-X_train_m)/X_train_sd

for k in range(nfeatures):
    X_test_m = np.mean(X_3D_test[:,:,k])
    X_test_sd = np.std(X_3D_test[:,:,k])
    X_3D_test[:,:,k] = (X_3D_test[:,:,k]-X_test_m)/X_test_sd

Now, we create the 1D convnet model. I have tried to tune the number of layers, number of filters in each layer, the batch size and the dropout ratio to give the best validation accuracy. Removing the 256 filters layer resulted in a very poor accuracy. Adding more layers resulted in overfitting (high accuracy on training data but low on validation). Lower dropout ratio resulted in over fitting too. I added layers one by one and observed how the training and validation accuracy change over training. 15% of the training data is kept for the validation. The high dropout ration is to avoid overfitting during training. Also, I tried to minimize the complexity of the model by introducing enough layers that can achieve the max possible accuracy with the current features created above.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
predict=np.zeros([y_test_new.shape[0],y_test_new.shape[1],3])
for k in range(1):
  verbose, epochs, batch_size = 1, 600, 512
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=8, activation='relu', input_shape=(ntimestamp,nfeatures)))
  model.add(Dropout(0.5))
  model.add(Conv1D(filters=128, kernel_size=8, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Conv1D(filters=256, kernel_size=8, activation='relu'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=1))
  model.add(Flatten())
  model.add(Dense(128, activation='relu')) 
  model.add(Dropout(0.5))
  model.add(Dense(y_train_new.shape[1], activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
  model.fit(X_3D_train, y_train_new, epochs=epochs, batch_size=batch_size, verbose=verbose)


We calculate the accuracy by comparing the prediction vs ytest:



In [ ]:
# evaluate model
predict=np.zeros([y_test_new.shape[0],y_test_new.shape[1],3])

yhat=model.predict(X_3D_test)
predict[:,:,k]=yhat

#_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

yhat=np.mean(predict,axis=2)
yhat_final = np.array(list(np.argmax(yhat,axis=1)))
y_testt=np.argmax(y_test_new,axis=1)

accuracy=np.mean(yhat_final==y_testt)
print('The accuracy is:', accuracy)